In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 12))

import random

import pycamhd.lazycache as lazycache
import pycamhd.lazyqt as lazyqt

## Local imports
from timer import Timer

In [2]:
filename = '/RS03ASHS/PN03B/06-CAMHDA301/2016/07/24/CAMHDA301-20160724T030000Z.mov'
num_frames = 10

def random_frames( n_frames, max_frame = 25000 ):
    random.seed(5)   # I want this to be _non_-random
    return random.sample(range(1,max_frame), n_frames )


def test_lazycache( url ):
    repo = lazycache.lazycache(url)
    return test_repo( repo )
    
def test_lazyqt( path ):
    repo = lazyqt.repo( path )
    return test_repo( repo )
    
def test_repo( repo ):
    times = []
    for i in random_frames(num_frames, 30):
        with Timer() as t:
            repo.get_frame( filename, i )
        times.append(t)
    
    return times


local_cached   = 'http://localhost:8080'
local_uncached = "http://localhost:9080"


# Direct LazyQT conversion

### Direct disk access

In [3]:
times = test_lazyqt("/Users/aaron/workspace/CamHD_analysis/data")
msec = [t.msecs for t in times]
print("Local, noncached: %f ms per frame" % np.mean(msec))

Local, noncached: 29.693532 ms per frame


### Local HTTP server 

In [4]:
times = test_lazyqt("http://localhost:9081/")   ## Local Nginx server as defined in the docker-compose configuration
msec = [t.msecs for t in times]
print("Local, noncached: %f ms per frame" % np.mean(msec))

Local, noncached: 68.099809 ms per frame


### Contacting CI directly 

In [5]:
times = test_lazyqt("https://rawdata.oceanobservatories.org/files/")
msec = [t.msecs for t in times]
print("Local, noncached: %f ms per frame" % np.mean(msec))

Local, noncached: 2990.657091 ms per frame


# Local lazycache server, non-caching

### Using local disk

In [7]:
times = test_lazycache("%s/v1/berna/data/" % local_uncached )

msec = [t.msecs for t in times]
print("Local, noncached: %f ms per frame" % np.mean(msec))

Local, noncached: 552.746105 ms per frame


### connecting to local HTTP server

In [8]:
times = test_lazycache("%s/v1/nginx_data/" % local_uncached )

msec = [t.msecs for t in times]
print("Nginx, noncached: %f ms per frame" % np.mean(msec))

Nginx, noncached: 511.022305 ms per frame


### contacting CI directly

In [9]:
times = test_lazycache("%s/v1/org/oceanobservatories/rawdata/files/" % local_uncached )

msec = [t.msecs for t in times]
print("CI, noncached: %f ms per frame" % np.mean(msec))

CI, noncached: 590.296125 ms per frame


# Local Lazycache server, with cacheing enabled

### operating on local data

In [10]:
times = test_lazycache("%s/v1/berna/data/" % local_cached )

msec = [t.msecs for t in times]
print("Local, cached: %f ms per frame" % np.mean(msec))

Local, cached: 443.422079 ms per frame


### connecting to local Nginx server

In [11]:
times = test_lazycache("%s/v1/nginx_data/" % local_cached )

msec = [t.msecs for t in times]
print("Nginx, cached: %f ms per frame" % np.mean(msec))

Nginx, cached: 482.806802 ms per frame


### contacting CI directly

In [12]:
times = test_lazycache("%s/v1/org/oceanobservatories/rawdata/files/" % local_cached )

msec = [t.msecs for t in times]
print("CI, cached: %f ms per frame" % np.mean(msec))

CI, cached: 453.917098 ms per frame


# Google App Engine instance of Lazycache

## Without cache

In [15]:
repo = lazycache.lazycache( 'https://camhd-app-dev-nocache.appspot.com/v1/org/oceanobservatories/rawdata/files' )

times = test_repo(repo)

msec = [t.msecs for t in times]
print("GAE, no cache: %f ms per frame" % np.mean(msec))

GAE, no cache: 524.795985 ms per frame


### With cache

In [16]:
repo = lazycache.lazycache()   ## Default repo

times = test_repo(repo)

msec = [t.msecs for t in times]
print("GAE, cached: %f ms per frame" % np.mean(msec))

GAE, cached: 1457.211065 ms per frame
